In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from photutils.detection import DAOStarFinder
from astropy.wcs import WCS
from astroquery.vizier import Vizier

from astropy.table import Table
from astropy.units import Quantity
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia
from astropy import units as u

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


# Calculating photometric colors for model stars

### Goals:
- Learn to read in text files

- Master advanced plotting skills

- Create astropy tables

- Calculate integrated photometry

Here you will take a set of templates of stellar spectra and calculate AB photometry for a variety of optical and IR filters.

We start with a couple of functions that you will need.

## Exercise 1: Reading in and plotting stellar spectra

First you will need to load in the spectral templates. They are located in the directory called PICKLES_STELLARTEMPS, from this paper: https://ui.adsabs.harvard.edu/abs/1998PASP..110..863P/abstract. You can try using glob to find the names of the *.txt iles within pickles_stellartemps.

from glob import glob

spectra = glob('pickles_stellartemps/*.dat')

print(spectra)

#### Load each one separately. As a challenge, you could try writing a loop that adds each spectrum to a dictionary which can be accessed later. Plot the spectra on one loglog plot. The units of wavelength are Angstrom, and the units of flux are arbitrary F_lambda.

stellar_dict = {} # initialize dictionary 

wave1, spectrum1 = [open your file here] # open the file

stellar_dict['your label'] = {'wave': wave1, 'spectrum': spectrum1} # label this element of the dictionary and repeat for each stellar type


and so on!

In [2]:
# Load in all stellar spectra here

## Exercise 2: Reading in and plotting photometric filters


Use the SVO filter profile service http://svo2.cab.inta-csic.es/theory/fps/index.php?mode=browse&gname=Subaru&asttype= to find the transmission curves for the Hubble Space Telescope, the Subaru Telescope, and the GAIA and 2MASS missions. Specifically, we want HST WFC3 F125W and F160W, Subaru Suprime g r i and z, GAIA g, bp, and rp and 2MASS J and H. 


Load each one and plot them on a single plot and label each filter with a different color. Challenge: try using plt.fill_between() to fill in the filters, and plt.text() to label them directly on the plot (i.e. not in a legend.)

In [3]:
# Load in transmission curves

## Now glue them together: overlay the photometric filters on top of your stellar spectra on one plot.

In [ ]:
# Your plot here

## Exercise 3: Extrapolating stellar spectra to the RJ tail

Stars are blackbodies that peak, generally speaking, in the UV or optical regimes. But the Rayleigh-Jeans tail extends into the infrared. If we wanted to calculate infrared colors of these stars, we could approximate the RJ tail as a power law. 

When fitting a power law, this is a STRAIGHT LINE in loglog space. That means:

$$ y = ax^ b $$
$$ log(y) = log(a) + b * log(x) $$

Now we can just do a linear regression on the long wavelength end of the spectrum. Try using the linregress function from scipy.stats. https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html

Fit a power law to each spectrum and plot each out to 2 microns. Plot the original spectrum with the corresponding power law as a dotted line. Keep in mind what form of this equation you will need in order to plot it in a log plot.


In [ ]:
# your linear regression here


In [ ]:
# your plot here

## Exercise 4: Calculating integrated photometry

When calculating the flux within a photometric band, we have to take into account the transmission function of the filter itself. You can read more here: https://arxiv.org/abs/astro-ph/0502120. This will give you info about calculating magnitudes in all kinds of units!


The average flux density per WAVELENGTH through a photometric filter is:

$$ <F_{\lambda}> = \frac{\int{\lambda F_{\lambda}(\lambda) S(\lambda) d\lambda}}{\int{\lambda S(\lambda) d\lambda}}  $$

where $F_{\lambda}$ is an object's spectrum at the wavelengths of the filter and $S(\lambda)$ is the transmission curve.

This is NOT the same as flux density per FREQUENCY, but one can be converted to the other.

The magnitude is 

$$ m = -2.5 \rm log(F_{\nu}) + zeropoint $$

where the zeropoint will depend on your photometric system. Here we will use AB mags, where the zeropoint is 8.926 Jy.

Fill in the blanks to build your photometry function.

In [ ]:
def AB_mags(spectrum_wave, spectrum_flux, filter_wave, transmission, fnu=True):
    '''
    Spectrum wave, spectrum flux = wavelength and flux array of your spectrum
    Filter wave, transmission = wavelength array and transmission function for a filter
    This function is different depending on if your spectrum is in Fnu or Flambda units, so
    make sure to check the units of your spectrum. HINT: The ones I've given you are in Flambda. :)
    '''
    
    # Step 1: interpolate your transmission function
    phot_filter = ### your interpolation function here
    
    # Step 2: Define the limits of your spectrum where you will calculate photometry
    limits = np.where((spectrum_wave< ???? )\
                        &(spectrum_wave>= ???? ))
    
    wave_filter = spectrum_wave[limits]
    
    flux_filter = spectrum_flux[limits]
     
    # Step 3: Calculate the values of the transmission for the interpolated grid.
    filter_response = []
    for angstrom in wave_filter:
        new_trans = ???
        filter_response.append(new_trans)
        
    
    c = 3.0e18 # speed of light in AA/s
    
    if fnu:
    # This is assuming input wavelength is in Angstrom and input flux is in Fnu; need to convert to Flambda if so
        Flambda = c / wave_filter**2 * flux_filter
    else:
        Flambda = flux_filter
        
    # Step 4: Calculate the integral. What are the arguments from the equation above?
    integrand_bottom = ???? 
    integrand_top = ????
    
    # Hint: Try using np.trapz for the integration.
    average_flux_lambda = ????
    
    # Step 5: Photometric flux is given in units of Jansky, which is a function of NU, not LAMBDA. I'll do it for you :)
    
    # Tokunaga+Vacca05 Eqn. A11 --> calculate pivot wavelength to convert <Flambda> to <Fnu>
    lambda_pivot_integrand_top = wave_filter * filter_response
    lambda_pivot_integrand_bottom = filter_response / wave_filter
    # Lambda^2 = the effective central wavelength, helpful to know for plotting purposes
    lambda_pivot2 = np.trapz(lambda_pivot_integrand_top, wave_filter) / np.trapz(lambda_pivot_integrand_bottom, wave_filter)
    
    average_flux_nu = 1e23 * lambda_pivot2 / c * average_flux_lambda # in units of Jy
    
    # Step 6: Calculate the AB magnitude from the definition of Jy. 
    
    m = ?????
    
    return m, average_flux_nu, np.sqrt(lambda_pivot2)

### Let's test out your function on a spectrum of Vega (located in this directory). Plot the spectrum of Vega. In another panel, plot the g, r, i, y, and z colors from Subaru. Come grab a moderator to check your photometry!

In [ ]:
# your vega spectrum here

## Exercise 5: Building a table of stellar colors.

Because the spectral templates are in arbitrary flux density units, their absolute magnitudes won't make much sense. However, we can still calculate their colors, which is just the difference in magnitudes between two filters! Practice calculating some optical colors like g-r or B-V. Use your photometry function on the infrared portion too and calculate J-H. 


Create an astropy table to keep track of the spectral type and the various colors. The documentation lives here: https://docs.astropy.org/en/stable/table/construct_table.html

Challenge: create a loop that adds a row of information for each spectral type.

In [4]:
# your table here

# Example table initialization:
# stellar_table = Table(names=['spectral type', 'B mag'], dtype=('S3', 'float'))
# row = ['A0V', '14']
# stellar_table.add_row(row)
# How would you add more columns or change the initialization to include more columns to begin with?

